In [1]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import openai
import argparse
import os
from langchain.agents import initialize_agent
from langchain.llms import OpenAI
from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory

import openai
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
import shutil
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
from langchain.chains import ConversationalRetrievalChain
embedding = OpenAIEmbeddings()
import re
from paperqa import Docs

In [3]:
top_shap = list(np.load(f'./data/top_shap_features.npy',allow_pickle=True))
print(top_shap
      )

['volume_per_atom_0', 'symmetry_function_G1_19', 'unoccupied_energy_levels_CBM_4', 'distance_matrix_3', 'symmetry_function_G1_14']


In [6]:
keyword_search = 'metal organic frameworks open metal sites'
papers = paperscraper.search_papers(keyword_search, limit=10)

AttributeError: module 'paperscraper' has no attribute 'search_papers'

In [2]:
lit_directory = './data/arxiv_downloads/'
docs_list = [f'{lit_directory}/{doc}' for doc in os.listdir(lit_directory)]
qa_docs = Docs()
for d in docs_list:
    qa_docs.add(d)


In [4]:
answer = qa_docs.query("What is a MOF?")

RuntimeError: This event loop is already running

In [6]:
llm = ChatOpenAI(
            temperature=0,
            model_name="gpt-3.5-turbo-0613",
            request_timeout=1000)
    
memory = ConversationBufferMemory(memory_key="chat_history")
readonlymemory = ReadOnlySharedMemory(memory=memory)

db1 = Chroma(persist_directory="./data/chroma/", 
                    embedding_function=embedding)
retriever1 = db1.as_retriever()
chain1 =  ConversationalRetrievalChain.from_llm(llm, 
                                                retriever=retriever1, 
                                                memory=readonlymemory,
                                                   )

In [3]:
top_shap = list(np.load(f'./data/top_shap_features.npy',allow_pickle=True))

In [11]:
for ft in top_shap:
    feature = ' '.join(ft.split('_')[:-1])
    query = f"""What is the AUC of the model?
    
        """

    response = chain1.run(query)
    print(response)

The AUC of the model is 0.876713103626608.
The AUC of the model is 0.876713103626608.
The AUC of the model is 0.876713103626608.


In [7]:
response = chain1.run("what is the AUC'?")
print(response)

There is no mention of AUC' in the provided context. Therefore, I don't have enough information to answer your question.


In [ ]:
def _explain_lime(data_path,label,model_path,
                 top_k=5,savedir='./data',
                 mode='classification',num_samples=100):
   
   weights = []

   df = pd.read_csv(data_path,header=0)
   ## use all data for the lime analysis
   df_x = df.drop(label,axis = 1)

   model = xgb.XGBClassifier()
   model.load_model(model_path)

   if mode=='classification': class_names=[0,1]
   else: class_names=[f'{label}']

   explainer = LimeTabularExplainer(df_x.values, 
                                    feature_names=list(df_x.columns), 
                                    class_names=class_names,
                                    mode=mode)
   

   df_sample = df_x.sample(num_samples)
   for i in range(len(df_sample)):
        exp = explainer.explain_instance(df_sample.iloc[i], model.predict_proba, 
        num_features=700, top_labels=True)
        exp_map = exp.as_map()
        cls = list(exp_map.keys())[0]
        #sort weights otherwise they are printed from highest to lowest w
        exp_map[cls].sort(key=lambda x: x[0])
        ws = np.array(exp_map[cls])[:,-1]
        zscore = stats.zscore(ws)
        weights.append(zscore)
        
   global_w = np.sum(np.vstack(weights),axis=0)

   ## get top k features from LIME values
   top_fts = np.array(df_x.columns)[abs(global_w).argsort()[-top_k:][::-1]]
   lime = global_w[abs(global_w).argsort()[-top_k:][::-1]]
   y_pos = np.arange(top_k)

   ## plot LIME analysis 
   fig, ax = plt.subplots()
   ax.barh(y_pos, lime)
   ax.set_yticks(y_pos, labels=top_fts )
   ax.invert_yaxis()  # labels read top-to-bottom
   ax.set_xlabel('Z-score lime values')
   ax.set_ylabel('Features')
   fig.savefig(f'{savedir}/lime_bar_top_{top_k}.png',
               bbox_inches='tight', dpi=300)
   
   ## write summary of LIME analysis
   summary = f'To explain the model behavior, LIME explanations were generated. \
    Please note these are global observations based on {num_samples} data points.'
   
   for ft, l in zip(top_fts,lime):
        summary+= f"Feature {ft} has an average \
             z-score of {l} with towards the prediction."
        
   prompt = f"Summarize and write an brief of the model behavior \
    from the following: {summary}"
   lime_summary = get_response(prompt)

   #save summary
   with open(f'{savedir}/lime_summary.txt',"w+") as f: 
       f.write(lime_summary)
       f.close()

   #update vectordb
   vector_db(lit_directory = f'{savedir}/lime_summary.txt')

   np.save(f'{savedir}/top_{top_k}_lime_fts.npy',top_fts)

   return top_fts
  
